In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

### 1.1.6. Fibonacci with a generator

In [2]:
from typing import Generator
def fib6(n: int) -> Generator[int, None, None]:
    yield 0
    if n > 0:
        last: int = 0
        next: int = 1
        for _ in range(1, n):
            last, next = next, last + next
            yield next                

In [3]:
%time
for i in fib6(50):
    print(i)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 11.4 µs
0
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765
10946
17711
28657
46368
75025
121393
196418
317811
514229
832040
1346269
2178309
3524578
5702887
9227465
14930352
24157817
39088169
63245986
102334155
165580141
267914296
433494437
701408733
1134903170
1836311903
2971215073
4807526976
7778742049
12586269025


### 1.2. Trivial compaction

In [4]:
import sys

In [5]:
x = 1100000

In [6]:
sys.getsizeof(x)

28

In [7]:
class CompressedGene:
    def __init__(self, gene:str) -> None:
        self._compress(gene)
    def _compress(self, gene:str) -> None:
        self.bit_string: int = 1
        for nucleotide in gene.upper():
            self.bit_string <<= 2
            if nucleotide == "A":
                self.bit_string |= 0b00
            elif nucleotide == "C":
                self.bit_string |= 0b01
            elif nucleotide == "G":
                self.bit_string |= 0b10
            elif nucleotide == "T":
                self.bit_string |= 0b11
            else:
                raise ValueError(f'Invalid Nucleotide: {nucleotide}')
    def decompress(self) -> str:
        gene: str = ''
        for i in range(self.bit_string.bit_length() - 1, 2):
            bits: int = self.bit_string >> i & 0b11
            if bits == 0b00:
                gene += "A"
            elif bits == 0b01:
                gene += "C"
            elif bits == 0b10:
                gene += "G"
            elif bits == 0b11:
                gene += "T"
            else:
                raise ValueError(f'Invalid bits: {bits}')
        return gene[::-1] 
            
                 

In [8]:
original_seq = "ACTACGACGCAGATAGACAGTAGACGATA" * 100
sys.getsizeof(original_seq)

2949

In [9]:
compressed: CompressedGene = CompressedGene(original_seq)
sys.getsizeof(compressed.bit_string)

800

In [10]:
sys.getsizeof(compressed.decompress())

49

### 1.3. Unbreakable Criptografy

In [11]:
from secrets import token_bytes
from typing import Tuple

In [12]:
def random_key(length: int) -> int:
    tb: bytes = token_bytes(length)
    return int.from_bytes(tb, "big")

In [13]:
random_key(4)

846015917

In [14]:
token_bytes(6)

b'\x92\xd5\x9d7\xfc\x8d'

In [15]:
def encrypt(original: str) -> Tuple[int, int]:
    original_bytes: bytes = original.encode()
    dummy: int = random_key(len(original_bytes))
    original_key: int = int.from_bytes(original_bytes, 'big')
    encrypted: int = original_key ^ dummy # XOR
    return dummy, encrypted        

In [16]:
def decrypt(key1: int, key2: int) -> str:
    decrypted: int = key1 ^ key2 # XOR
    temp: bytes = decrypted.to_bytes((decrypted.bit_length() + 7) // 8, 'big')
    return temp.decode()
                                     

In [17]:
key1, key2 =  encrypt('One Time Pad!')

In [18]:
decrypt(key1, key2)

'One Time Pad!'

### 1.4. Calculating pi

In [19]:
def calculate_pi(n_terms: int) -> float:
    numerator: float = 4.0
    denominator: float = 1.0
    operation: float = 1.0
    pi: float = 0.0
    for _ in range(n_terms):
        pi += operation * (numerator / denominator)
        denominator += 2.0
        operation *= -1.0
    return pi

In [20]:
%time calculate_pi(1000000)

CPU times: user 167 ms, sys: 158 µs, total: 167 ms
Wall time: 165 ms


3.1415916535897743

### 1.5. Hanoi Towers

In [21]:
from typing import TypeVar, Generic, List
T = TypeVar('T')

class Stack(Generic[T]):
    def __init__(self) -> None:
        self._container: List[T] = []
    
    @property
    def empty(self) -> bool:
        return not self._container
    
    def push(self, item: T) -> None:
        self._container.append(item)
    
    def pop(self) -> None:
        return self._container.pop() # LIFO
    
    def __repr__(self) -> None:
        return repr(self._container)
    

In [22]:
def hanoi(begin: Stack[int], end: Stack[int], temp: Stack[int], n:int) -> None:
    if n == 1:
        end.push(begin.pop())
    else:
        hanoi(begin, temp, end, n - 1)
        hanoi(begin, end, temp, 1)
        hanoi(temp, end, begin, n - 1)

In [23]:
for n in range(3, 25):
    num_discs: int = n
    tower_a: Stack[int] = Stack()
    tower_b: Stack[int] = Stack()
    tower_c: Stack[int] = Stack()

    for i in range(1, num_discs + 1):
        tower_a.push(i)

    %time hanoi(tower_a, tower_b, tower_c, num_discs)


CPU times: user 14 µs, sys: 1 µs, total: 15 µs
Wall time: 18.8 µs
CPU times: user 19 µs, sys: 2 µs, total: 21 µs
Wall time: 23.6 µs
CPU times: user 28 µs, sys: 2 µs, total: 30 µs
Wall time: 33.4 µs
CPU times: user 49 µs, sys: 4 µs, total: 53 µs
Wall time: 55.8 µs
CPU times: user 94 µs, sys: 9 µs, total: 103 µs
Wall time: 105 µs
CPU times: user 179 µs, sys: 16 µs, total: 195 µs
Wall time: 196 µs
CPU times: user 464 µs, sys: 0 ns, total: 464 µs
Wall time: 470 µs
CPU times: user 724 µs, sys: 0 ns, total: 724 µs
Wall time: 727 µs
CPU times: user 1.46 ms, sys: 0 ns, total: 1.46 ms
Wall time: 1.46 ms
CPU times: user 2.91 ms, sys: 0 ns, total: 2.91 ms
Wall time: 2.91 ms
CPU times: user 6.52 ms, sys: 0 ns, total: 6.52 ms
Wall time: 6.28 ms
CPU times: user 12.5 ms, sys: 0 ns, total: 12.5 ms
Wall time: 12.2 ms
CPU times: user 27.2 ms, sys: 0 ns, total: 27.2 ms
Wall time: 26.8 ms
CPU times: user 55.2 ms, sys: 0 ns, total: 55.2 ms
Wall time: 54.7 ms
CPU times: user 108 ms, sys: 0 ns, total: 108 ms

## Chapter 2 - Search Problems
### 2.1. Storing DNA

In [24]:
from enum import IntEnum
from typing import Tuple, List

In [25]:
Nucleotide: IntEnum = IntEnum('Nucleotide', ('A', 'C', 'G', 'T'))
Codon =  Tuple[Nucleotide, Nucleotide, Nucleotide]
Gene =  List[Codon]

In [26]:
Nucleotide(1)
Nucleotide(2)
Nucleotide(3)
Nucleotide(4)

<Nucleotide.A: 1>

<Nucleotide.C: 2>

<Nucleotide.G: 3>

<Nucleotide.T: 4>

In [27]:
Codon

typing.Tuple[__main__.Nucleotide, __main__.Nucleotide, __main__.Nucleotide]

In [28]:
gene_str: str = 'ACGTGGCTCTCTAACGTACGTACGGGGTTTATATATACCCTAGGACTCCCTTT'

In [29]:
def string_to_gene(s: str) -> Gene:
    gene: Gene = []
    for i in range(0, len(s), 3):
        if (i + 2) >= len(s):
            return gene
        codon: Codon = (Nucleotide[s[i]], Nucleotide[s[i+1]], Nucleotide[s[i+2]])
        gene.append(codon)
    return gene

In [30]:
my_gene: Gene = string_to_gene(gene_str)

In [31]:
my_gene

[(<Nucleotide.A: 1>, <Nucleotide.C: 2>, <Nucleotide.G: 3>),
 (<Nucleotide.T: 4>, <Nucleotide.G: 3>, <Nucleotide.G: 3>),
 (<Nucleotide.C: 2>, <Nucleotide.T: 4>, <Nucleotide.C: 2>),
 (<Nucleotide.T: 4>, <Nucleotide.C: 2>, <Nucleotide.T: 4>),
 (<Nucleotide.A: 1>, <Nucleotide.A: 1>, <Nucleotide.C: 2>),
 (<Nucleotide.G: 3>, <Nucleotide.T: 4>, <Nucleotide.A: 1>),
 (<Nucleotide.C: 2>, <Nucleotide.G: 3>, <Nucleotide.T: 4>),
 (<Nucleotide.A: 1>, <Nucleotide.C: 2>, <Nucleotide.G: 3>),
 (<Nucleotide.G: 3>, <Nucleotide.G: 3>, <Nucleotide.G: 3>),
 (<Nucleotide.T: 4>, <Nucleotide.T: 4>, <Nucleotide.T: 4>),
 (<Nucleotide.A: 1>, <Nucleotide.T: 4>, <Nucleotide.A: 1>),
 (<Nucleotide.T: 4>, <Nucleotide.A: 1>, <Nucleotide.T: 4>),
 (<Nucleotide.A: 1>, <Nucleotide.C: 2>, <Nucleotide.C: 2>),
 (<Nucleotide.C: 2>, <Nucleotide.T: 4>, <Nucleotide.A: 1>),
 (<Nucleotide.G: 3>, <Nucleotide.G: 3>, <Nucleotide.A: 1>),
 (<Nucleotide.C: 2>, <Nucleotide.T: 4>, <Nucleotide.C: 2>),
 (<Nucleotide.C: 2>, <Nucleotide.C: 2>, 

In [32]:
def linear_contains(gene: Gene, key_codon: Codon) -> bool:
    for codon in gene:
        if codon == key_codon:
            return True
    return False

In [33]:
acg: Codon = (Nucleotide.A, Nucleotide.C, Nucleotide.G)
gat: Codon = (Nucleotide.G, Nucleotide.A, Nucleotide.T)

In [34]:
print(linear_contains(my_gene, acg))
print(linear_contains(my_gene, gat))

True
False


### 2.1. Binary Search

In [35]:
def binary_contains(gene: Gene, key_codon: Codon) -> bool:
    low: int = 0
    high: int = len(gene) - 1
    while low <= high:
        mid: int =  (low + high) // 2
        if gene[mid] < key_codon:
            low = mid + 1
        elif gene[mid] > key_codon:
            high = mid - 1
        else:
            return True
    return False        

In [36]:
my_sorted_gene: Gene = sorted(my_gene)
my_sorted_gene

[(<Nucleotide.A: 1>, <Nucleotide.A: 1>, <Nucleotide.C: 2>),
 (<Nucleotide.A: 1>, <Nucleotide.C: 2>, <Nucleotide.C: 2>),
 (<Nucleotide.A: 1>, <Nucleotide.C: 2>, <Nucleotide.G: 3>),
 (<Nucleotide.A: 1>, <Nucleotide.C: 2>, <Nucleotide.G: 3>),
 (<Nucleotide.A: 1>, <Nucleotide.T: 4>, <Nucleotide.A: 1>),
 (<Nucleotide.C: 2>, <Nucleotide.C: 2>, <Nucleotide.T: 4>),
 (<Nucleotide.C: 2>, <Nucleotide.G: 3>, <Nucleotide.T: 4>),
 (<Nucleotide.C: 2>, <Nucleotide.T: 4>, <Nucleotide.A: 1>),
 (<Nucleotide.C: 2>, <Nucleotide.T: 4>, <Nucleotide.C: 2>),
 (<Nucleotide.C: 2>, <Nucleotide.T: 4>, <Nucleotide.C: 2>),
 (<Nucleotide.G: 3>, <Nucleotide.G: 3>, <Nucleotide.A: 1>),
 (<Nucleotide.G: 3>, <Nucleotide.G: 3>, <Nucleotide.G: 3>),
 (<Nucleotide.G: 3>, <Nucleotide.T: 4>, <Nucleotide.A: 1>),
 (<Nucleotide.T: 4>, <Nucleotide.A: 1>, <Nucleotide.T: 4>),
 (<Nucleotide.T: 4>, <Nucleotide.C: 2>, <Nucleotide.T: 4>),
 (<Nucleotide.T: 4>, <Nucleotide.G: 3>, <Nucleotide.G: 3>),
 (<Nucleotide.T: 4>, <Nucleotide.T: 4>, 

In [37]:
%time binary_contains(my_sorted_gene, acg)
%time linear_contains(my_sorted_gene, acg)

%time binary_contains(my_sorted_gene, gat)
%time linear_contains(my_sorted_gene, gat)

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 10.5 µs


True

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 12.4 µs


True

CPU times: user 12 µs, sys: 0 ns, total: 12 µs
Wall time: 17.2 µs


False

CPU times: user 18 µs, sys: 0 ns, total: 18 µs
Wall time: 26 µs


False

### 2.1.4 Generic example

In [38]:
from __future__ import annotations
from typing_extensions import Protocol
from typing import TypeVar, Iterable, Sequence, Generic, List, Callable, Set, Deque, Dict, Any, Optional
from heapq import heappush, heappop

In [39]:
T = TypeVar('T')

def linear_contains(iterable: Iterable[T], key: T) -> bool:
    for item in iterable:
        if item == key:
            return True
    return False

C = TypeVar('C', bound='Comparable')

In [40]:
class Comparable(Protocol):
    
    def __eq__(self, other: Any) -> bool:
        ...
    
    def __lt__(self: C, other: C) -> bool:
        ...
    
    def __gt__(self: C, other: C) -> bool:
        return(not self < other) and self != other
    
    def __le__(self: C, other: C) -> bool:
        return self < other or self == other
    
    def __ge__(self: C, other: C) -> bool:
        return not self < other

def binary_contains(sequence: Sequence[C], key: C) -> bool:
    low: int = 0
    high: int = len(sequence) - 1
    while low <= high:
        mid: int =  (low + high) // 2
        if sequence[mid] < key:
            low = mid + 1
        elif sequence[mid] > key:
            high = mid - 1
        else:
            return True
    return False            

In [41]:
%time linear_contains([1, 5, 15, 15, 15, 15, 20], 5)
%time binary_contains([1, 5, 15, 15, 15, 15, 20], 5)

%time linear_contains(['a', 'd', 'e', 'f', 'z'], 'f')
%time binary_contains(['a', 'd', 'e', 'f', 'z'], 'f')


%time linear_contains(['John', 'Mark', 'Ronald', 'Sarah'], 'Sheila')
%time binary_contains(['John', 'Mark', 'Ronald', 'Sarah'], 'Sheila')


CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 8.58 µs


True

CPU times: user 10 µs, sys: 0 ns, total: 10 µs
Wall time: 15.3 µs


True

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 11.9 µs


...

True

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 11.4 µs


False

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 13.4 µs


False

### 2.2 Labyrinths

In [42]:
from enum import Enum
from typing import List, NamedTuple, Callable, Optional
import random
from math import sqrt
#from generic_search import dfs, bfs, node_to_path, astar, Node

class Cell(str, Enum):
    EMPTY = " "
    BLOCKED = "X"
    START = "S"
    GOAL = "G"
    PATH = "*"

class MazeLocation(NamedTuple):
    row: int
    column: int

In [43]:
class Maze:
    def __init__(self, 
                 rows: int = 10, 
                 columns: int = 10, 
                 sparseness: float = 0.2, start: MazeLocation = MazeLocation(0, 0), goal: MazeLocation = MazeLocation(9, 9)) -> None:
        # initialize basic instance variables
        self._rows: int = rows
        self._columns: int = columns
        self.start: MazeLocation = start
        self.goal: MazeLocation = goal
        # fill the grid with empty cells
        self._grid: List[List[Cell]] = [[Cell.EMPTY for c in range(columns)] for r in range(rows)]
        # populate the grid with blocked cells
        self._randomly_fill(rows, columns, sparseness)
        # fill the start and goal locations in
        self._grid[start.row][start.column] = Cell.START
        self._grid[goal.row][goal.column] = Cell.GOAL

    def _randomly_fill(self, rows: int, columns: int, sparseness: float):
        for row in range(rows):
            for column in range(columns):
                if random.uniform(0, 1.0) < sparseness:
                    self._grid[row][column] = Cell.BLOCKED

    # return a nicely formatted version of the maze for printing
    def __str__(self) -> str:
        output: str = ""
        for row in self._grid:
            output += "".join([c.value for c in row]) + "\n"
        return output
    
    def goal_test(self, ml: MazeLocation) -> bool:
        return ml == self.goal
    
    def sucessors(self, ml: MazeLocation) -> List[MazeLocation]:
        locations: List[MazeLocation] = []
        if ml.row + 1 < self._rows and self._grid[ml.row + 1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row + 1, ml.column))
        if ml.row - 1 >= 0 and self._grid[ml.row - 1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row - 1, ml.column))
        if ml.column + 1 < self._columns and self._grid[ml.row][ml.column + 1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column + 1))
        if ml.column - 1 >= 0 and self._grid[ml.row][ml.column - 1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column - 1))
        return locations
    
    def mark(self, path: List[MazeLocation]):
        for maze_location in path:
            self._grid[maze_location.row][maze_location.column] = Cell.PATH        
        self._grid[self.start.row][self.start.column] = Cell.START
        self._grid[self.goal.row][self.goal.column] = Cell.GOAL
    
    def clear(self, path: List[MazeLocation]):
        for maze_location in path:
            self._grid[maze_location.row][maze_location.column] = Cell.EMPTY        
        self._grid[self.start.row][self.start.column] = Cell.START
        self._grid[self.goal.row][self.goal.column] = Cell.GOAL

In [44]:
maze: Maze = Maze()
print(maze)

S   X  X  
 X X X    
 X X X X  
X     X X 
          
   X      
         X
 X X   XXX
 X        
 X X X XXG



### 2.2.3. Depth-First Search (DFS)

In [45]:
class Node(Generic[T]):
    def __init__(self, state: T, parent: Optional[Node], cost: float=0.0, heuristic: float=0.0) -> None:
        self.state: T = state
        self.parent: Optional[Node] = parent
        self.cost: float = cost
        self.heuristic: float = heuristic
    
    def __lt__(self, other:Node) -> bool:
        return (self.cost + self.heuristic) < (other.cost + other.heuristic)

def dfs(initial: T, goal_test: Callable[[T], bool], sucessors: Callable[[T], List[T]]) -> Optional[Node[T]]:
    frontier: Stack[Node[T]] = Stack()
    frontier.push(Node(initial, None))
    explored: Set[T] = {initial}
    
    while not frontier.empty:
        current_node: Node[T] = frontier.pop()
        current_state: T = current_node.state
        
        if goal_test(current_state):
            return current_node
        
        for child in sucessors(current_state):
            if child in explored:
                continue
            explored.add(child)
            frontier.push(Node(child, current_node))
    return None

In [46]:
def node_to_path(node: Node[T]) -> List[T]:
    path: List[T] = [node.state]
    
    while node.parent is not None:
        node = node.parent
        path.append(node.state)
    
    path.reverse()
    return path

In [47]:
### Running the maze process
m: Maze = Maze()
print(m)

S     X   
          
      XX  
 X XX    X
  X   X XX
          
   X   X X
      X   
  X       
    X   XG



In [48]:
solution1: Optional[Node[MazeLocation]] = dfs(m.start, m.goal_test, m.sucessors)
if solution1 is None:
    print('No solution found using depth-first search!')
else:
    path1: List[MazeLocation] = node_to_path(solution1)
    m.mark(path1)
    print(m)
    m.clear(path1)

S*****X   
     **** 
      XX* 
 X XX****X
  X***X XX
****  *** 
*  X **X*X
******X **
  X      *
    X   XG



### 2.2.4. Breadth-First Search (BFS)

In [49]:
class Queue(Generic[T]):
    def __init__(self) -> None:
        self._container: Deque[T] = Deque()
    
    @property
    def empty(self) -> Bool:
        return not self._container
    
    def push(self, item: T) -> None:
        self._container.append(item)
    
    def pop(self) -> T:
        return self._container.popleft() # FIFO
    
    def __repr__(self) -> str:
        return repr(self._container)


In [50]:
def bfs(initial: T, goal_test: Callable[[T], bool], successors: Callable[[T], List[T]]) -> Optional[Node[T]]:
    # frontier is where we've yet to go
    frontier: Queue[Node[T]] = Queue()
    frontier.push(Node(initial, None))
    # explored is where we've been
    explored: Set[T] = {initial}

    # keep going while there is more to explore
    while not frontier.empty:
        current_node: Node[T] = frontier.pop()
        current_state: T = current_node.state
        # if we found the goal, we're done
        if goal_test(current_state):
            return current_node
        # check where we can go next and haven't explored
        for child in successors(current_state):
            if child in explored:  # skip children we already explored
                continue
            explored.add(child)
            frontier.push(Node(child, current_node))
    return None  # went through everything and never found goal

In [51]:
solution2: Optional[Node[MazeLocation]] = bfs(m.start, m.goal_test, m.sucessors)
if solution2 is None:
    print('No solution found using BFS!')
else:
    path2: List[MazeLocation] = node_to_path(solution2)
    m.mark(path2)
    print(m)
    m.clear(path2)

S     X   
*         
*     XX  
*X XX    X
* X   X XX
*         
*  X   X X
****  X   
  X*******
    X   XG



### 2.2.5. Astar search

In [52]:
class PriorityQueue(Generic[T]):
    def __init__(self) -> None:
        self._container: List[T] = []

    @property
    def empty(self) -> bool:
        return not self._container  # not is true for empty container

    def push(self, item: T) -> None:
        heappush(self._container, item)  # in by priority

    def pop(self) -> T:
        return heappop(self._container)  # out by priority

    def __repr__(self) -> str:
        return repr(self._container)

In [53]:
def euclidean_distance(goal: MazeLocation) -> Callable[[MazeLocation], float]:
    def distance(ml: MazeLocation) -> float:
        xdist: int = ml.column - goal.column
        ydist: int = ml.row - goal.row
        return sqrt((xdist * xdist) + (ydist * ydist))
    return distance

In [54]:
location = MazeLocation(3,3)
euclidean_distance(location)

<function __main__.euclidean_distance.<locals>.distance(ml: 'MazeLocation') -> 'float'>

In [55]:
def manhattan_distance(goal: MazeLocation) -> Callable[[MazeLocation], float]:
    def distance(ml: MazeLocation) -> float:
        xdist: int = abs(ml.column - goal.column)
        ydist: int = abs(ml.row - goal.row)
        return (xdist + ydist)
    return distance

In [56]:
def astar(initial: T, goal_test: Callable[[T], bool], successors: Callable[[T], List[T]], heuristic: Callable[[T], float]) -> Optional[Node[T]]:
    # frontier is where we've yet to go
    frontier: PriorityQueue[Node[T]] = PriorityQueue()
    frontier.push(Node(initial, None, 0.0, heuristic(initial)))
    # explored is where we've been
    explored: Dict[T, float] = {initial: 0.0}

    # keep going while there is more to explore
    while not frontier.empty:
        current_node: Node[T] = frontier.pop()
        current_state: T = current_node.state
        # if we found the goal, we're done
        if goal_test(current_state):
            return current_node
        # check where we can go next and haven't explored
        for child in successors(current_state):
            new_cost: float = current_node.cost + 1  # 1 assumes a grid, need a cost function for more sophisticated apps

            if child not in explored or explored[child] > new_cost:
                explored[child] = new_cost
                frontier.push(Node(child, current_node, new_cost, heuristic(child)))
    return None  # went through everything and never found goal

### Testing A*

In [57]:
distance: Callable[[MazeLocation], float] = manhattan_distance(m.goal)
solution3: Optional[Node[MazeLocation]] =  astar(m.start, m.goal_test, m.sucessors, distance)

if solution3 is None:
    print('No solution found using A*!')
else:
    path3: List[MazeLocation] =  node_to_path(solution3)
    m.mark(path3)
    print(m)

S     X   
*         
*     XX  
*X XX    X
**X   X XX
 ******** 
   X   X*X
      X **
  X      *
    X   XG



## 2.3. Missionaries and Canibals problem

In [58]:
from __future__ import annotations
from typing import List, Optional

In [59]:
MAX_NUM: int = 3

In [60]:
class MCState:
    def __init__(self, missionaries: int, cannibals: int, boat: bool) -> None:
        self.wm: int = missionaries # west bank missionaries
        self.wc: int = cannibals # west bank cannibals
        self.em: int = MAX_NUM - self.wm  # east bank missionaries
        self.ec: int = MAX_NUM - self.wc  # east bank cannibals
        self.boat: bool = boat

    def __str__(self) -> str:
        return ("On the west bank there are {} missionaries and {} cannibals.\n" 
                "On the east bank there are {} missionaries and {} cannibals.\n"
                "The boat is on the {} bank.")\
            .format(self.wm, self.wc, self.em, self.ec, ("west" if self.boat else "east"))

    def goal_test(self) -> bool:
        return self.is_legal and self.em == MAX_NUM and self.ec == MAX_NUM

    @property
    def is_legal(self) -> bool:
        if self.wm < self.wc and self.wm > 0:
            return False
        if self.em < self.ec and self.em > 0:
            return False
        return True

    def successors(self) -> List[MCState]:
        sucs: List[MCState] = []
        if self.boat: # boat on west bank
            if self.wm > 1:
                sucs.append(MCState(self.wm - 2, self.wc, not self.boat))
            if self.wm > 0:
                sucs.append(MCState(self.wm - 1, self.wc, not self.boat))
            if self.wc > 1:
                sucs.append(MCState(self.wm, self.wc - 2, not self.boat))
            if self.wc > 0:
                sucs.append(MCState(self.wm, self.wc - 1, not self.boat))
            if (self.wc > 0) and (self.wm > 0):
                sucs.append(MCState(self.wm - 1, self.wc - 1, not self.boat))
        else: # boat on east bank
            if self.em > 1:
                sucs.append(MCState(self.wm + 2, self.wc, not self.boat))
            if self.em > 0:
                sucs.append(MCState(self.wm + 1, self.wc, not self.boat))
            if self.ec > 1:
                sucs.append(MCState(self.wm, self.wc + 2, not self.boat))
            if self.ec > 0:
                sucs.append(MCState(self.wm, self.wc + 1, not self.boat))
            if (self.ec > 0) and (self.em > 0):
                sucs.append(MCState(self.wm + 1, self.wc + 1, not self.boat))
        return [x for x in sucs if x.is_legal]

In [61]:
def display_solution(path: List[MCState]):
    if len(path) == 0: # sanity check
        return
    old_state: MCState = path[0]
    print(old_state)
    for current_state in path[1:]:
        if current_state.boat:
            print("{} missionaries and {} cannibals moved from the east bank to the west bank.\n"
                  .format(old_state.em - current_state.em, old_state.ec - current_state.ec))
        else:
            print("{} missionaries and {} cannibals moved from the west bank to the east bank.\n"
                  .format(old_state.wm - current_state.wm, old_state.wc - current_state.wc))
        print(current_state)
        old_state = current_state

In [62]:
start: MCState = MCState(MAX_NUM, MAX_NUM, True)
solution: Optional[Node[MCState]] = bfs(start, MCState.goal_test, MCState.successors)
if solution is None:
    print("No solution found!")
else:
    path: List[MCState] = node_to_path(solution)
    display_solution(path)

On the west bank there are 3 missionaries and 3 cannibals.
On the east bank there are 0 missionaries and 0 cannibals.
The boat is on the west bank.
0 missionaries and 2 cannibals moved from the west bank to the east bank.

On the west bank there are 3 missionaries and 1 cannibals.
On the east bank there are 0 missionaries and 2 cannibals.
The boat is on the east bank.
0 missionaries and 1 cannibals moved from the east bank to the west bank.

On the west bank there are 3 missionaries and 2 cannibals.
On the east bank there are 0 missionaries and 1 cannibals.
The boat is on the west bank.
0 missionaries and 2 cannibals moved from the west bank to the east bank.

On the west bank there are 3 missionaries and 0 cannibals.
On the east bank there are 0 missionaries and 3 cannibals.
The boat is on the east bank.
0 missionaries and 1 cannibals moved from the east bank to the west bank.

On the west bank there are 3 missionaries and 1 cannibals.
On the east bank there are 0 missionaries and 2 c

## 3. Constraint-Satisfaction Problems

### 3.1. Building a CSP framework

In [63]:
from typing import Generic, TypeVar, Dict, List, Optional
from abc import ABC, abstractmethod

V = TypeVar('V') # variable type
D = TypeVar('D') # domain type


# Base class for all constraints
class Constraint(Generic[V, D], ABC):
    # The variables that the constraint is between
    def __init__(self, variables: List[V]) -> None:
        self.variables = variables

    # Must be overridden by subclasses
    @abstractmethod
    def satisfied(self, assignment: Dict[V, D]) -> bool:
        ...

In [64]:
class CSP(Generic[V, D]):
    def __init__(self, variables: List[V], domains: Dict[V, List[D]]) -> None:
        self.variables: List[V] = variables # variables to be constrained
        self.domains: Dict[V, List[D]] = domains # domain of each variable
        self.constraints: Dict[V, List[Constraint[V, D]]] = {}
        for variable in self.variables:
            self.constraints[variable] = []
            if variable not in self.domains:
                raise LookupError("Every variable should have a domain assigned to it.")

    def add_constraint(self, constraint: Constraint[V, D]) -> None:
        for variable in constraint.variables:
            if variable not in self.variables:
                raise LookupError("Variable in constraint not in CSP")
            else:
                self.constraints[variable].append(constraint)
    
    # Check if the value assignment is consistent by checking all constraints
    # for the given variable against it
    def consistent(self, variable: V, assignment: Dict[V, D]) -> bool:
        for constraint in self.constraints[variable]:
            if not constraint.satisfied(assignment):
                return False
        return True

    def backtracking_search(self, assignment: Dict[V, D] = {}) -> Optional[Dict[V, D]]:
        # assignment is complete if every variable is assigned (our base case)
        if len(assignment) == len(self.variables):
            return assignment

        # get all variables in the CSP but not in the assignment
        unassigned: List[V] = [v for v in self.variables if v not in assignment]

        # get the every possible domain value of the first unassigned variable
        first: V = unassigned[0]
        for value in self.domains[first]:
            local_assignment = assignment.copy()
            local_assignment[first] = value
            # if we're still consistent, we recurse (continue)
            if self.consistent(first, local_assignment):
                result: Optional[Dict[V, D]] = self.backtracking_search(local_assignment)
                # if we didn't find the result, we will end up backtracking
                if result is not None:
                    return result
        return None

In [65]:
from typing import Dict, List, Optional


class MapColoringConstraint(Constraint[str, str]):
    def __init__(self, place1: str, place2: str) -> None:
        super().__init__([place1, place2])
        self.place1: str = place1
        self.place2: str = place2

    def satisfied(self, assignment: Dict[str, str]) -> bool:
        # If either place is not in the assignment then it is not
        # yet possible for their colors to be conflicting
        if self.place1 not in assignment or self.place2 not in assignment:
            return True
        # check the color assigned to place1 is not the same as the
        # color assigned to place2
        return assignment[self.place1] != assignment[self.place2]

In [66]:
variables: List[str] = ["Western Australia", "Northern Territory", "South Australia",
                        "Queensland", "New South Wales", "Victoria", "Tasmania"]
domains: Dict[str, List[str]] = {}
for variable in variables:
    domains[variable] = ["red", "green", "blue"]
csp: CSP[str, str] = CSP(variables, domains)
csp.add_constraint(MapColoringConstraint("Western Australia", "Northern Territory"))
csp.add_constraint(MapColoringConstraint("Western Australia", "South Australia"))
csp.add_constraint(MapColoringConstraint("South Australia", "Northern Territory"))
csp.add_constraint(MapColoringConstraint("Queensland", "Northern Territory"))
csp.add_constraint(MapColoringConstraint("Queensland", "South Australia"))
csp.add_constraint(MapColoringConstraint("Queensland", "New South Wales"))
csp.add_constraint(MapColoringConstraint("New South Wales", "South Australia"))
csp.add_constraint(MapColoringConstraint("Victoria", "South Australia"))
csp.add_constraint(MapColoringConstraint("Victoria", "New South Wales"))
csp.add_constraint(MapColoringConstraint("Victoria", "Tasmania"))
solution: Optional[Dict[str, str]] = csp.backtracking_search()
if solution is None:
    print("No solution found!")
else:
    print(solution)

{'Western Australia': 'red', 'Northern Territory': 'green', 'South Australia': 'blue', 'Queensland': 'red', 'New South Wales': 'green', 'Victoria': 'red', 'Tasmania': 'green'}


### 3.3. The eight Queens Problem

In [67]:
columns: List[int] = [1,2,3,4,5,6,7,8]
rows: Dict[int, List[int]] = {}
for column in columns:
    rows[column] = [1,2,3,4,5,6,7,8]
csp: CSP[int, int] = CSP(columns, rows)

In [68]:
csp.variables

[1, 2, 3, 4, 5, 6, 7, 8]

In [69]:
csp.domains

{1: [1, 2, 3, 4, 5, 6, 7, 8],
 2: [1, 2, 3, 4, 5, 6, 7, 8],
 3: [1, 2, 3, 4, 5, 6, 7, 8],
 4: [1, 2, 3, 4, 5, 6, 7, 8],
 5: [1, 2, 3, 4, 5, 6, 7, 8],
 6: [1, 2, 3, 4, 5, 6, 7, 8],
 7: [1, 2, 3, 4, 5, 6, 7, 8],
 8: [1, 2, 3, 4, 5, 6, 7, 8]}

In [70]:
csp.constraints

{1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: []}

In [71]:
from typing import Dict, List, Optional

class QueensConstraint(Constraint[int, int]):
    def __init__(self, columns: List[int]) -> None:
        super().__init__(columns)
        self.columns: List[int] = columns

    def satisfied(self, assignment: Dict[int, int]) -> bool:
        # q1c = queen 1 column, q1r = queen 1 row
        for q1c, q1r in assignment.items():
            # q2c = queen 2 column
            for q2c in range(q1c + 1, len(self.columns) + 1):
                if q2c in assignment:
                    q2r: int = assignment[q2c] # q2r = queen 2 row
                    if q1r == q2r: # same row?
                        return False
                    if abs(q1r - q2r) == abs(q1c - q2c): # same diagonal?
                        return False
        return True # no conflict

In [72]:
QueensConstraint(columns).columns

[1, 2, 3, 4, 5, 6, 7, 8]

In [73]:
csp.add_constraint(QueensConstraint(columns))
solution: Optional[Dict[int, int]] =  csp.backtracking_search()
if solution is None:
    print('No solution found!')
else:
    print(solution)

{1: 1, 2: 5, 3: 8, 4: 6, 5: 3, 6: 7, 7: 2, 8: 4}


### 3.4. Word search


In [74]:
from random import choice
from string import ascii_uppercase

In [75]:
Grid = List[List[str]]  # type alias for grids


class GridLocation(NamedTuple):
    row: int
    column: int


def generate_grid(rows: int, columns: int) -> Grid:
    # initialize grid with random letters
    return [[choice(ascii_uppercase) for c in range(columns)] for r in range(rows)]


def display_grid(grid: Grid) -> None:
    for row in grid:
        print("".join(row))


def generate_domain(word: str, grid: Grid) -> List[List[GridLocation]]:
    domain: List[List[GridLocation]] = []
    height: int = len(grid)
    width: int = len(grid[0])
    length: int = len(word)
    for row in range(height):
        for col in range(width):
            columns: range = range(col, col + length)
            rows: range = range(row, row + length)
            if col + length <= width:
                # left to right
                domain.append([GridLocation(row, c) for c in columns])
                # diagonal towards bottom right
                if row + length <= height:
                    domain.append([GridLocation(r, col + (r - row)) for r in rows])
            if row + length <= height:
                # top to bottom
                domain.append([GridLocation(r, col) for r in rows])
                # diagonal towards bottom left
                if col - length >= 0:
                    domain.append([GridLocation(r, col - (r - row)) for r in rows])
    return domain

In [76]:
grid: Grid = generate_grid(9, 9)
grid

[['M', 'D', 'A', 'L', 'N', 'V', 'A', 'P', 'S'],
 ['F', 'M', 'E', 'X', 'A', 'Y', 'I', 'R', 'B'],
 ['A', 'P', 'H', 'P', 'I', 'E', 'T', 'P', 'S'],
 ['D', 'V', 'K', 'X', 'H', 'J', 'V', 'O', 'V'],
 ['H', 'Z', 'T', 'V', 'W', 'T', 'F', 'J', 'G'],
 ['H', 'M', 'Z', 'C', 'A', 'V', 'O', 'L', 'Q'],
 ['B', 'Z', 'N', 'Z', 'D', 'Y', 'X', 'M', 'N'],
 ['T', 'H', 'N', 'W', 'H', 'P', 'G', 'Q', 'J'],
 ['P', 'R', 'W', 'J', 'K', 'O', 'U', 'N', 'G']]

In [77]:
display_grid(grid)

MDALNVAPS
FMEXAYIRB
APHPIETPS
DVKXHJVOV
HZTVWTFJG
HMZCAVOLQ
BZNZDYXMN
THNWHPGQJ
PRWJKOUNG


In [78]:
class WordSearchConstraint(Constraint[str, List[GridLocation]]):
    def __init__(self, words: List[str]) -> None:
        super().__init__(words)
        self.words: List[str] = words

    def satisfied(self, assignment: Dict[str, List[GridLocation]]) -> bool:
        # if there are any duplicates grid locations then there is an overlap
        all_locations = [locs for values in assignment.values() for locs in values]
        return len(set(all_locations)) == len(all_locations)

In [79]:
Grid

typing.List[typing.List[str]]

In [80]:
grid: Grid = generate_grid(9, 9)
words: List[str] = ["MATTHEW", "JOE", "MARY", "SARAH", "SALLY"]
locations: Dict[str, List[List[GridLocation]]] = {}
for word in words:
    locations[word] = generate_domain(word, grid)
csp: CSP[str, List[GridLocation]] = CSP(words, locations)
csp.add_constraint(WordSearchConstraint(words))
solution: Optional[Dict[str, List[GridLocation]]] = csp.backtracking_search()
if solution is None:
    print("No solution found!")
else:
    for word, grid_locations in solution.items():
        # random reverse half the time
        if choice([True, False]):
            grid_locations.reverse()
        for index, letter in enumerate(word):
            (row, col) = (grid_locations[index].row, grid_locations[index].column)
            grid[row][col] = letter
    display_grid(grid)


MATTHEWJY
SARAHYFOR
IGVPBLPEA
LLYTGLNEM
GQRLMAULV
EYBLGSMGY
RINYGDYNJ
ABQPJEFVS
KAOZYBUZY


### 3.5. SEND+MORE=MONEY

In [81]:
class SendMoreMoneyConstraint(Constraint[str, int]):
    def __init__(self, letters: List[str]) -> None:
        super().__init__(letters)
        self.letters: List[str] = letters

    def satisfied(self, assignment: Dict[str, int]) -> bool:
        # if there are duplicate values then it's not a solution
        if len(set(assignment.values())) < len(assignment):
            return False

        # if all variables have been assigned, check if it adds correctly
        if len(assignment) == len(self.letters):
            s: int = assignment["S"]
            e: int = assignment["E"]
            n: int = assignment["N"]
            d: int = assignment["D"]
            m: int = assignment["M"]
            o: int = assignment["O"]
            r: int = assignment["R"]
            y: int = assignment["Y"]
            send: int = s * 1000 + e * 100 + n * 10 + d
            more: int = m * 1000 + o * 100 + r * 10 + e
            money: int = m * 10000 + o * 1000 + n * 100 + e * 10 + y
            return send + more == money
        return True # no conflict

In [82]:
letters: List[str] = ["S", "E", "N", "D", "M", "O", "R", "Y"]
possible_digits: Dict[str, List[int]] = {}
for letter in letters:
    possible_digits[letter] = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
possible_digits["M"] = [1]  # so we don't get answers starting with a 0
csp: CSP[str, int] = CSP(letters, possible_digits)
csp.add_constraint(SendMoreMoneyConstraint(letters))
solution: Optional[Dict[str, int]] = csp.backtracking_search()
if solution is None:
    print("No solution found!")
else:
    print(solution)

{'S': 9, 'E': 5, 'N': 6, 'D': 7, 'M': 1, 'O': 0, 'R': 8, 'Y': 2}


## 4. Graph Problems

In [83]:
from dataclasses import dataclass

@dataclass
class Edge:
    u: int # the "from" vertex
    v: int # the "to" vertex

    def reversed(self) -> Edge:
        return Edge(self.v, self.u)

    def __str__(self) -> str:
        return f"{self.u} -> {self.v}"

In [84]:
e = Edge(2, 4)

In [87]:
e.reversed()

Edge(u=4, v=2)

In [91]:
e.__str__()

'2 -> 4'

In [92]:
V = TypeVar('V') # type of the vertices in the graph


class Graph(Generic[V]):
    def __init__(self, vertices: List[V] = []) -> None:
        self._vertices: List[V] = vertices
        self._edges: List[List[Edge]] = [[] for _ in vertices]

    @property
    def vertex_count(self) -> int:
        return len(self._vertices) # Number of vertices

    @property
    def edge_count(self) -> int:
        return sum(map(len, self._edges)) # Number of edges

    # Add a vertex to the graph and return its index
    def add_vertex(self, vertex: V) -> int:
        self._vertices.append(vertex)
        self._edges.append([]) # add empty list for containing edges
        return self.vertex_count - 1 # return index of added vertex

    # This is an undirected graph,
    # so we always add edges in both directions
    def add_edge(self, edge: Edge) -> None:
        self._edges[edge.u].append(edge)
        self._edges[edge.v].append(edge.reversed())

    # Add an edge using vertex indices (convenience method)
    def add_edge_by_indices(self, u: int, v: int) -> None:
        edge: Edge = Edge(u, v)
        self.add_edge(edge)

    # Add an edge by looking up vertex indices (convenience method)
    def add_edge_by_vertices(self, first: V, second: V) -> None:
        u: int = self._vertices.index(first)
        v: int = self._vertices.index(second)
        self.add_edge_by_indices(u, v)

    # Find the vertex at a specific index
    def vertex_at(self, index: int) -> V:
        return self._vertices[index]

    # Find the index of a vertex in the graph
    def index_of(self, vertex: V) -> int:
        return self._vertices.index(vertex)

    # Find the vertices that a vertex at some index is connected to
    def neighbors_for_index(self, index: int) -> List[V]:
        return list(map(self.vertex_at, [e.v for e in self._edges[index]]))

    # Lookup a vertice's index and find its neighbors (convenience method)
    def neighbors_for_vertex(self, vertex: V) -> List[V]:
        return self.neighbors_for_index(self.index_of(vertex))

    # Return all of the edges associated with a vertex at some index
    def edges_for_index(self, index: int) -> List[Edge]:
        return self._edges[index]

    # Lookup the index of a vertex and return its edges (convenience method)
    def edges_for_vertex(self, vertex: V) -> List[Edge]:
        return self.edges_for_index(self.index_of(vertex))

    # Make it easy to pretty-print a Graph
    def __str__(self) -> str:
        desc: str = ""
        for i in range(self.vertex_count):
            desc += f"{self.vertex_at(i)} -> {self.neighbors_for_index(i)}\n"
        return desc